# Lab 11. Obiekty `bytes`, `bytearray` oraz moduł `threading`

## 1. Wbudowane funkcje `bytes` oraz `bytearray`

Wbudowana metoda `bytes([source[, encoding[, errors]]])` zwraca niemutowalną sekwencję wartości całkowitoliczbowych w zakresie 0 <=x < 256.

Jej mutowalna wersja to metoda `bytearray()`.

In [64]:
bytes(), bytes('ala', 'utf-8'), bytes(b'\xff'), bytes(3), bytes([1,2,3,9,9,10,12])

(b'', b'ala', b'\xff', b'\x00\x00\x00', b'\x01\x02\x03\t\t\n\x0c')

**Dlaczego takie wartości na wyjściu?**

Dla bajtów o kodach od 32 do 126 wypisywany jest znak ASCII.

Dla bajtów odpowiadającym znakom tabulacji, nowego wiersza, powrtou karetki i znaku \ odpowiednio \t,\n,\r oraz \\.

Dla sekwencji bajtowej pojawiają się obydwa delimitery łańcuchów (czyli ' oraz ") to cała sekwencja jest ograniczana znakami ' (domyślny delimiter w Pythonie), a dowolny znak ' wewnątrz łańcucha jest zapisywany jako \'.

Dla każdej innej wartości bajta używana jest szesnastkowa sekwencja ucieczki np. \x00, który widzimy w komórce wyjściowej powyżej, bajt null.

In [53]:
print(bytes.__doc__)

bytes(iterable_of_ints) -> bytes
bytes(string, encoding[, errors]) -> bytes
bytes(bytes_or_buffer) -> immutable copy of bytes_or_buffer
bytes(int) -> bytes object of size given by the parameter initialized with null bytes
bytes() -> empty bytes object

Construct an immutable array of bytes from:
  - an iterable yielding integers in range(256)
  - a text string encoded using the specified encoding
  - any object implementing the buffer API.
  - an integer


In [56]:
# jeżeli nie poprzedzamy łańcucha znaków znakiem b, to aby z obiektu str stworzyć obiekt bytes należy podać również kodowanie
sentence = "Ala ma kota, który ma 5 lat."
bytes(sentence, 'utf-8')

b'Ala ma kota, kt\xc3\xb3ry ma 5 lat.'

In [51]:
bytes(sentence)

TypeError: string argument without an encoding

In [59]:
# lub
sentence.encode('utf-8')

b'Ala ma kota, kt\xc3\xb3ry ma 5 lat.'

In [58]:
b'Ala ma kota, kt\xc3\xb3ry ma 5 lat.'.decode()

'Ala ma kota, który ma 5 lat.'

In [38]:
int.from_bytes(b'\xff')

255

In [41]:
# powyżej to zapis heksadecymalny (szesnastkowy)
# wyjaśnienie wartości powyżej
15 * 16**1 + 15 * 16**0

255

In [40]:
binary = b'111'

print(binary)
print(bin(int(binary)))
print(int(binary))
print(hex(int(binary)))
print(int(binary, base=2))
print(int(binary, base=8))
print(int(binary, base=16))

b'111'
0b1101111
111
0x6f
7
73
273


In [70]:
byte = b'\x01\x00\x00'
print(int.from_bytes(byte, 'little'))
print(int.from_bytes(byte, 'big'))

1
65536


In [25]:
# wyjaśnienie
# little endian
print(0 * 256**2 + 0 * 256**1 + 1 * 256**0)
# big endian
1 * 256**2 + 0 * 256**1 + 0 * 256**0

1


65536

In [76]:
# to faktycznie lista liczb
print(bytes([1,2,3,9,9,10,12]))
list(bytes([1,2,3,9,9,10,12]))

b'\x01\x02\x03\t\t\n\x0c'


[1, 2, 3, 9, 9, 10, 12]

Przykład faktycznych problemów, które można napotkać przy kodowaniu znaków w Pythonie.

In [79]:
text = 'café'
bytes(text, 'utf_8')

b'caf\xc3\xa9'

In [88]:
print(text)

café


In [89]:
# plik jest zapisywany ze wskazaniem kodowania znaków
with open('file.bin', 'w', encoding='utf-8') as file:
    print(f'Obiekt pliku: {file}')
    file.write(text)

Obiekt pliku: <_io.TextIOWrapper name='file.bin' mode='w' encoding='utf-8'>


In [90]:
# plik jest wczytywany bez podanie encodowania, wybrane zostało systemowe (uruchomiona na systemie Windows 10)
with open('file.bin', 'r') as file:
    print(f'Obiekt pliku: {file}')
    print(file.read())

Obiekt pliku: <_io.TextIOWrapper name='file.bin' mode='r' encoding='cp1250'>
cafĂ©


**Wbudowane funkcje `ord` oraz `chr`**

In [ ]:
# funkcja ord zwraca numer przekazanego znaku z tablicy unicode
ord('a'), ord('<'), ord('1')

In [ ]:
# funkcja chr zwraca znak z podanej pozycji tablic unicode
chr(97), chr(60), chr(49)

In [ ]:
# a co zwrawca ord po przekazaniu wartości typu byte?
ord(b'\x02'), ord(b'\xff')

In [ ]:
chr(int(ord('\x02'))), chr(int(ord('\xaa')))

In [ ]:
int('0x02c', 16)

In [ ]:
[(i, chr(i)) for i in range(0,241)]

In [12]:
chr(34)

'"'

To jeszcze przykład nieco z innej beczki, ale w praktyce bardzo przydatny. Otóż domyślny sposób sortowania w Pythonie nie za bardzo będzie nam pasował jak chcemy sortować łańcuchy znaków, które zawierają inne wartości nic znaki ASCII. Przykład poniżej.

In [101]:
words = ['a', 'ą', 'ć', 'c', 'e', 'ę']
sorted(words)

['a', 'c', 'e', 'ą', 'ć', 'ę']

In [104]:
import locale

# moduł locale po pierwsze pozwala nam na ustawienie LC_COLLATE, za dokumentacją:
# locale.LC_COLLATE
# Locale category for sorting strings. The functions strcoll() and strxfrm() of the locale module are affected.

locale.setlocale(locale.LC_COLLATE, 'pl_PL.utf-8')

'pl_PL.utf-8'

In [106]:
# teraz trzeba jeszcze przekazań klucz sortowania, który jest funkcją biorącą pod uwagę ustawienia
# ze zmiennej locale.LC_COLLATE
sorted(words, key=locale.strxfrm)

['a', 'ą', 'c', 'ć', 'e', 'ę']

## 2. Współbieżność/równoległość w Pythonie

**Równoległość** to możliwość wykonywania wielu zadań w tym samym czasie, co możliwe jest dzięki procesorom wielordzeniowym, wielu procesorom lub wielu komputerowm w klastrze.

**Współbieżność (wielozadaniowość)** możliwość wykonywania wielu zadań poprzez przełączanie się między nimi co możliwe jest do zrealizowania również na jednordzeniowym procesorze.

**Proces** to samodzielnie działający program uruchomiony w systemie operacyjnym. Ma własną przestrzeń adresową (pamięć), zmienne, zasoby i działa niezależnie od innych procesów. Procesy komunikują się ze sobą przez specjalne mechanizmy (np. IPC). W Pythonie, aby procesy mogły się ze sobą komunikować to dane muszą być przesylane między nimi w formie bajtów, więc konieczna jest wcześniejsza serializacja, które jest dość kosztowna.

**Wątek (ang. thread)** to lżejsza jednostka wykonawcza działająca w obrębie procesu. Wątki tego samego procesu współdzielą pamięć i zasoby, ale mają własny licznik instrukcji i stos. Wątki są używane do wykonywania wielu zadań równolegle w ramach jednego procesu.



### 2.1 Moduł `threading`

> Dokumentacja:
> * https://docs.python.org/3/library/threading.html


W tym module głównym jego elementem jest klasa `Thread` (https://docs.python.org/3/library/threading.html#threading.Thread), która dostarcza następującą funkcjonalność:

* metoda `start()` - uruchamia zadanie wątku, może być wywołana co najwyżej raz. Umożliwia wykonanie metody `run()` obiektu w oddzielnym wątku,
* metoda `run()` - metoda reprezentująca główną aktywność wątku (czyli funkcję, która została przekazana jako ta, która ma zostać uruchomiona poza głównym wątkiem),
* metoda `join(timeout=None)` - jej wywołanie nakazuje głównemu wątkowi poczekać na zakończenia działania tego wątku przed wznowieniem działania wątku głównego,
* metoda `is_alive()` - zwraca informacje czy wątek jest wciąż aktywny.

Poza tym klasa posiada jeszcze kilka atrybutów, które nie są bardzo istotne.

_**Przykład 1**_

In [1]:
# przykład ze strony z dokumentacją modułu threading, lekko zmodyfikowany:
# https://docs.python.org/3/library/threading.html

import threading
import time
import random

def crawl(link, delay=3):
    print(f"crawl started for {link} with delay {delay}")
    time.sleep(delay)  # Blocking I/O (simulating a network request)
    print(f"crawl ended for {link}")

links = [
    "https://python.org",
    "https://docs.python.org",
    "https://peps.python.org",
]

# Start threads for each link
threads = []
for link in links:
    # Using `args` to pass positional arguments and `kwargs` for keyword arguments
    t = threading.Thread(target=crawl, args=(link,), kwargs={"delay": random.randint(3, 6)})
    threads.append(t)

# Start each thread
for t in threads:
    t.start()

# Wait for all threads to finish
for t in threads:
    t.join()


print('Po wykonaniu wszystkich wątków')

crawl started for https://python.org with delay 4
crawl started for https://docs.python.org with delay 4
crawl started for https://peps.python.org with delay 3
crawl ended for https://peps.python.org
crawl ended for https://docs.python.org
crawl ended for https://python.org
Po wykonaniu wszystkich wątków


Jeżeli uruchomisz ten kod kilkukrotnie to prawdopodobnie zobaczysz różną kolejność wyświetlanych komunikatów.

Poniżej zostanie zaprezentowany inny przykład, który nieco bardziej namacalnie pokaże jak wygląda czas wykonania tego samego zadania dla jednego wątku vs. większa ilość wątków.

Scenariusz wygląda tak:
* deklarujemy funkcję, która ma obliczać sumę liczb z przekazanej jej listy,
* ta funkcja ma to robić wolno, tj. dodaliśmy oczekiwanie 0.1 s dla każdej kolejnej zsumowanej liczby,
* aby można było bez większych kombinacji przekazać wyniki między wątkami dodaliśmy zmienną typu `list` zadeklarowaną poza funkcją, do której dodajemy kolejne wyniki działania funkcji `slow_sum`,
* w kolejnych krokach tworzymy listę liczb, a następnie sumujemy ją w jednym (głównym) wątku oraz w kilku (tu będzie 5) i porównujemy czasy.

_**Przykład 2**_

In [124]:
# funkcja wykonująca obliczenia "wolno"
from numbers import Number
import time

def slow_sum(vals: list[Number], result_list: list) -> Number:

    delay = 0.1
    result = 0
    print(f'Sumuję {vals}')
    for num in vals:
        result = result + num
        time.sleep(delay)

    result_list.append(result)

In [117]:
# scenariusz obejmuje porównanie czasu wykonania obliczenia bez wykorzystania wątków oraz w kilku wątkach

nums = list(range(2, 101, 2))
results = []

start = time.perf_counter()
slow_sum(nums, results)
print(f"Czas wykonania: {time.perf_counter() - start}")
print(f"Wynik: {sum(results)}")

Sumuję [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]
Czas wykonania: 5.028755600011209
Wynik: 2550


In [118]:
# teraz tworzymy kilka wątków i dzielimy listę na kilka fragmentów, a następnie dla każdego wątku przekażemy jeden fragment
import itertools

# w tym przykładzie chcielibyśmy 4 wątki, więc dzielimy dane
# jednak ze względu na to, że podział może nie być równy to paczek może być 4 lub 5
batch_size = len(nums) // 4
threads = []
results = []

start = time.perf_counter() 

for batch in itertools.batched(nums, batch_size):
    t = threading.Thread(target=slow_sum, kwargs={"vals": batch, "result_list": results})
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print(f"Czas wykonania: {time.perf_counter()  - start}")
print(f"Wynik: {sum(results)}")

Sumuję (2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24)
Sumuję (26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48)
Sumuję (50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72)
Sumuję (74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96)
Sumuję (98, 100)
Czas wykonania: 1.2102835999976378
Wynik: 2550


Wynik jest taki sam, więc całość działa, ale różnica w czasie wykonania jest spora, choć dość przewidywalna. Dla jednego wątku całość wykonywała się około 5.02 s, a gdyby przemnożyć ilość elementów listy do zsumowania przez opóźnienie mamy 50 * 0.1 = 5s, więc to by się zgadzało. W drugim przypadku czas jaki został wyświetlony to około 1.21 sekundy co też jest dość oczywiste, gdyż maksymalna liczba elementów do przeliczenia dla jednego wątku to 12, a same operacje nie są kosztowne, więc ta wartość jest głównie wynikiem działania opóźnienia poprzez `time.sleep(0.1)`.

Widać jednak, że 5 wątków wykonało tę pracę szybciej, ale gdyby zsumować ten czas z każdego wątku, to zapewne byłby on dłuższy ze zwględu na narzut obsługi wątków.

Tę wątki jednak **nie wykonują się równolegle**!

Te wątki wykonują się **współbieżnie** w ramach jednego procesu i jednego procesora!

Wewnętrzny mechanizm obsługi wątków w Pythonie przełącza wątki domyślnie co 5 ms, aby żaden z nich nie "trzymał" blokady GIL w nieskończoność.
Można sprawdzić wartość tego interwału poprzez `sys.getswitchinterval()`. Można go również zmienić z poziomu kodu poprzez wywołanie `sys.setswitchinterval(s)`.

W kolejnych przykładach zostaną pokazane inne możliwości obliczeń współbieżnych oraz zrównoleglonych.

Ze względu na aktualną naturę Pythona (właściwie wielu najpopularniejszych implementacji, w tym CPython, której używamy), każde wystąpienie (instancja) interpretera Pythona jest pojedynczym procesem i jest ograniczone poprzez mechanizm GIL (ang. Global Interpreter Lock), który nie pozwala programom działać domyślnie na wielu rdzeniach jednocześnie. Nie oznacza to, że w Pythonie nie ma takich mechanizmów, ale nie jest to jego natywne zachowanie w przeciwieństwie do wielu innych języków programowania.

Są w Pythonie funkcje, które zwalniają blokadę GIL i należą do nich funkcje, które przeprowadzają dyskowe operacje wejścia-wyjścia, operacje sieciowe oraz ... `time.sleep()`. Również zewnętrzne biblioteki takie jak NumPy, SciPy, zlib dla najbardziej intensywnych obliczeń zwalniają tę blokadę.

Kod CPythona, w którym można zobaczyć flagi zdejmujące blokadę GIL dla funkjcji `time.sleep()` można znaleźć tu: https://github.com/python/cpython/blob/7ba1f75f3f02b4b50ac6d7e17d15e467afa36aac/Modules/timemodule.c#L1880-L1882

In [37]:
import sys

sys.getswitchinterval()

0.005

Teraz sprawdzimy jak wygląda czas wykonania dla pojedynczego wątku oraz wielu wątków, ale bez wykorzystania funkcji `time.sleep`.

_**Przykład 3**_

In [121]:
def just_sum(vals: list[Number], result_list: list) -> Number:

    result = 0
    print(f'Sumuję {vals}')
    for num in vals:
        result = result + num

    result_list.append(result)

In [122]:
# tylko główny wątek

nums = list(range(2, 101, 2))
results = []

start = time.perf_counter() 
just_sum(nums, results)
print(f"Czas wykonania: {time.perf_counter()  - start}")
print(f"Wynik: {sum(results)}")

Sumuję [2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72, 74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96, 98, 100]
Czas wykonania: 0.0004031999851576984
Wynik: 2550


In [123]:
# wiele wątków

batch_size = len(nums) // 4
threads = []
results = []

start = time.perf_counter() 

for batch in itertools.batched(nums, batch_size):
    t = threading.Thread(target=just_sum, kwargs={"vals": batch, "result_list": results})
    threads.append(t)

for t in threads:
    t.start()

for t in threads:
    t.join()

print(f"Czas wykonania: {time.perf_counter()  - start}")
print(f"Wynik: {sum(results)}")

Sumuję (2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22, 24)
Sumuję (26, 28, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48)
Sumuję (50, 52, 54, 56, 58, 60, 62, 64, 66, 68, 70, 72)
Sumuję (74, 76, 78, 80, 82, 84, 86, 88, 90, 92, 94, 96)
Sumuję (98, 100)
Czas wykonania: 0.004826899996260181
Wynik: 2550


Teraz widać, że kod wielowątkowy w tym przypadku jest znacznie wolniejszy (około 600% !), gdyż wykonuje się współbieżnie oraz sam koszt obsługi wątków dodaje niemały narzut. Jeżeli chcemy wykonać takie zadanie **równolegle** musimy wykorzystać inne moduły.

W Pythonie mamy moduł `multiprocessing`, `concurrent.futures` oraz `subprocess`, które pozwalają na uruchamianie wielu procesów z poziomu kodu Pythona.

_**Przykład 4**_

In [74]:
import psutil

def print_cpu_usage():
    # ta pętla jest nieskończona!
    while True:
        cpu_usage = psutil.cpu_percent(interval=1)
        print(f"CPU Usage: {cpu_usage}%", end='\r')
        print('',end="")
        time.sleep(5)


t1 = threading.Thread(target=print_cpu_usage)

t1.start()
# próba oczekiwania na zakończenie wątku przez 20 sekund, później powrót sterowania do głównego wątk
# ale ten wątek dalej będzie działał. Jeżeli zakomentujemy poniższą linię, sterowanie od razu zostanie
# przekazane do głównego wątka i możemy uruchamiać kolejne komórki z kodem. A wątek będzie działał w tle
# w obu przypadkach, gdyż nie ma wewnątrz żadnej logiki, która zakończy działanie metody print_cpu_usage osadzonej w tym wątku.
t1.join(timeout=20.0)

CPU Usage: 8.1%%

In [5]:
# można w tym czasie robić coś innego
print('Wątek główny nie jest blokowany!')

Wątek główny nie jest blokowany!


_**Przykład 5**_

In [6]:
# Jeżeli chcemy zaprojektować obsługę wątku tak, żeby można go było przerwać to musimy to zrobić samodzielnie
# nie ma wbudowanej metody, która kończy wątek. Tu wykorzystamy klasę threading.Event
import threading
import time


class CpuMonitorThread(threading.Thread):
    def __init__(self):
        super().__init__()
        self._stop_event = threading.Event()

    def stop(self):
        self._stop_event.set()

    def run(self):
        while not self._stop_event.is_set():
            cpu_usage = psutil.cpu_percent(interval=1)
            print(f"\rCPU Usage: {cpu_usage}%", end="")
            time.sleep(5)

In [7]:
monitor = CpuMonitorThread()
monitor.start()

CPU Usage: 7.1%

In [8]:
monitor.stop()
monitor.join()

Powyższy przykład wykorzystuje klasę `threading.Event`, która jest wykorzystywana do komunikacji między wątkami. W klasie `CpuMonitrThread`, która dziedziczy po `threading.Thread` mamy inicjalizator, który definiuje event o nazwie `_stop_event` oraz oraz metodę `stop()`, która jest swego rodzaju flagą, której włączenie (wartość True) powoduje, że w metodzie `run` nie wykona się już w nic w pętli i funkcja zakończy swoje działanie. Ostatecznie wywołanie `join()` przekaże sterowanie do głównego wątku.

In [107]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


### Zadania

**Zadanie 1**  

Wyjaśnij w postaci komentarza wyniki (dlaczego taka wartość) wykonania poniższego fragmentu kodu.

```python
binary = '111'

print(binary)
print(bin(int(binary)))
print(int(binary))
print(hex(int(binary)))
print(int(binary, base=2))
print(int(binary, base=8))
print(int(binary, base=16))
```

**Zadanie 2**  
Wykorzystaj moduł `this` (sprawdź jego kod źródłowy) i korzystając z umieszczonego tam słownika kodującego (sprawdź dostępne zmienne modułu `this`) napisz skrypt, który będzie kodował tym słownikiem wpisywane zdanie (przechwytuj z klawiatury). Wypisuj na konsoli zakodowane zdanie (zwróć uwagę, że słownik kodujący zawiera tylko znaki ASCII).

**Zadanie 3**

Wykorzystując moduł `threading` oraz `requests` napisz kod, który w odddzielnych wątkach odpyta 5 wybranych przez Ciebie stron interntetowych o ich zawartość (HTML) i zapisze tę zawartość do pliku (funkcja, która będzie uruchamiana w ramach wątku będzie robiła obie te operacje, nie w oddzielnych funkcjach). Wypisuj na wyjściu komunikat o zakończeniu pobierania, tak aby można było prześledzieć kolejność wykonania operacji.

**Zadanie 4**

Wykorzystując przykład z pomiarem obciążenia procesora (przykład 5) napisz podobne rozwiązanie, które będzie wyświetlało % wykorzystania pamięci RAM. 

**Zadanie 5**

Wykorzystując moduł threading napisz funkcję, która odczytuje zawartość pliku (z opóźnieniem jak w przykładzie z obliczaniem sumy liczb). Dodaj obsługę modułu tqdm, który ma wyświetlać postęp tego odczytu. Stwórz 3-4 wątki, które będą odczytywały różne pliki i śledź ich wykonanie.






